# Heat equation example

## Analytic problem formulation

We consider the heat equation on segment $[0, 1]$, with dissipation on both sides, heating (input) $u$ on the left side, and measurement (output) $\tilde{y}$ on the right side:
$$
\begin{align*}
    \partial_t z(x, t) & = \partial_{xx} z(x, t), & 0 < x < 1,\ t > 0, \\
    \partial_x z(0, t) & = z(0, t) - u(t), & t > 0 \\
    \partial_x z(1, t) & = -z(1, t), & t > 0, \\
    \tilde{y}(t) & = z(1, t), & t > 0.
\end{align*}
$$

## Semidiscretized formulation

By central finite differences on the equidistant mesh $0 = x_1 < x_2 < \ldots < x_n = 1$, we obtain the semidiscretized formulation:
$$
\begin{align*}
    \partial_t z_i(t) & = \frac{z_{i - 1}(t) - 2 z_i(t) + z_{i + 1}(t)}{h^2}, & i = 1, 2, 3, \ldots, n - 1, n, \\
    \frac{z_2(t) - z_0(t)}{2 h} & = z_1(t) - u(t), \\
    \frac{z_{n + 1}(t) - z_{n - 1}(t)}{2 h} & = -z_n(t), \\
    y(t) & = z_n(t),
\end{align*}
$$
where $h = \frac{1}{n - 1}$, $z_i(t)$ approximates $z(x_i, t)$, and $y(t)$ approximates $\tilde{y}(t)$.

Separating cases $i = 1$ and $i = n$ in the first equation, we find:
$$
\begin{align*}
    \partial_t z_1(t) & = -2 n (n - 1) z_1(t) + 2 (n - 1)^2 z_2(t) + 2 (n - 1) u(t), \\
    \partial_t z_i(t) & = (n - 1)^2 z_{i - 1}(t) - 2 (n - 1)^2 z_i(t) + (n - 1)^2 z_{i + 1}(t),
    & i = 2, 3, \ldots, n - 1, \\
    \partial_t z_n(t) & = (n - 1)^2 z_{n - 1}(t) - 2 n (n - 1)  z_n(t), \\
    y(t) & = z_n(t),
\end{align*}
$$

## Import modules

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pymor.discretizations.iosys import LTISystem
from pymor.reductors.bt import bt
from pymor.reductors.lti import irka

import logging
logging.getLogger('pymor.algorithms.gram_schmidt.gram_schmidt').setLevel(logging.ERROR)

## Assemble $A$, $B$,  and $C$

In [ ]:
n = 100  # dimension of the system

A = np.zeros((n, n))
a = n * (n - 1)
b = (n - 1) ** 2
A[0, 0] = -2 * a
A[0, 1] = 2 * b
for i in range(1, n - 1):
    A[i, i - 1] = b
    A[i, i] = -2 * b
    A[i, i + 1] = b
A[-1, -2] = 2 * b
A[-1, -1] = -2 * a

B = np.zeros((n, 1))
B[0, 0] = 2 * (n - 1)

C = np.zeros((1, n))
C[0, n - 1] = 1

## LTI system

In [ ]:
lti = LTISystem.from_matrices(A, B, C)

print('n = {}'.format(lti.n))
print('m = {}'.format(lti.m))
print('p = {}'.format(lti.p))

## System poles

In [ ]:
lti.compute_poles()
poles = lti._poles
fig, ax = plt.subplots()
ax.plot(poles.real, poles.imag, '.')
ax.set_title('System poles')
plt.show()

## Bode plot of the full model

In [ ]:
w = np.logspace(-2, 3, 100)
lti.bode(w)
fig, ax = LTISystem.mag_plot(lti)
ax.set_title('Bode plot of the full model')
plt.show()

## Hankel singular values

In [ ]:
lti.compute_sv_U_V('lyap')
fig, ax = plt.subplots()
ax.semilogy(range(1, len(lti._sv['lyap']) + 1), lti._sv['lyap'], '.-')
ax.set_title('Hankel singular values')
plt.show()

## Norms of the system

In [ ]:
print('H_2-norm of the full model:   {}'.format(lti.norm()))
print('H_inf-norm of the full model: {}'.format(lti.norm('Hinf')))

## Balanced Truncation (BT)

In [ ]:
r = 5
rom_bt, _, _ = bt(lti, r)

In [ ]:
print('H_2-norm of the BT ROM:       {}'.format(rom_bt.norm()))
print('H_inf-norm of the BT ROM:     {}'.format(rom_bt.norm('Hinf')))
err_bt = lti - rom_bt
print('H_2-error for the BT ROM:     {}'.format(err_bt.norm()))
print('H_inf-error for the BT ROM:   {}'.format(err_bt.norm('Hinf')))

## Bode plot of the full and BT reduced model

In [ ]:
rom_bt.bode(w)
fig, ax = LTISystem.mag_plot((lti, rom_bt))
ax.set_title('Bode plot of the full and BT reduced model')
plt.show()

## Bode plot of the BT error system

In [ ]:
err_bt.bode(w)
fig, ax = LTISystem.mag_plot(err_bt)
ax.set_title('Bode plot of the BT error system')
plt.show()

## Iterative Rational Krylov Algorithm (IRKA)

In [ ]:
sigma = np.logspace(-1, 3, r)
tol = 1e-4
maxit = 100
rom_irka, _, reduction_data_irka = irka(lti, r, sigma, tol=tol, maxit=maxit, verbose=True, compute_errors=True)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy([np.min(d) for d in reduction_data_irka['dist']], '.-')
ax.set_title('Distances between shifts in IRKA iterations')
plt.show()

In [ ]:
print('H_2-norm of the IRKA ROM:     {}'.format(rom_irka.norm()))
print('H_inf-norm of the IRKA ROM:   {}'.format(rom_irka.norm('Hinf')))
err_irka = lti - rom_irka
print('H_2-error for the IRKA ROM:   {}'.format(err_irka.norm()))
print('H_inf-error for the IRKA ROM: {}'.format(err_irka.norm('Hinf')))

## Bode plot of the full and IRKA reduced model

In [ ]:
rom_irka.bode(w)
fig, ax = LTISystem.mag_plot((lti, rom_irka))
ax.set_title('Bode plot of the full and IRKA reduced model')
plt.show()

## Bode plot of the IRKA error system

In [ ]:
err_irka.bode(w)
fig, ax = LTISystem.mag_plot(err_irka)
ax.set_title('Bode plot of the IRKA error system')
plt.show()